In [42]:
using BenchmarkTools
using LinearAlgebra

## Vecchia funzione

In [43]:
function r(args...)
    args = collect(args)
    n = length(args)
    if n == 1 # rotation in 2D
        angle = args[1]; COS = cos(angle); SIN = sin(angle)
        mat = Matrix{Float64}(LinearAlgebra.I, 3, 3)
        mat[1,1] = COS;    mat[1,2] = -SIN;
        mat[2,1] = SIN;    mat[2,2] = COS;
    end

     if n == 3 # rotation in 3D
        mat = Matrix{Float64}(LinearAlgebra.I, 4, 4)
        angle = norm(args);
        if norm(args) != 0.0
			axis = args #normalize(args)
			COS = cos(angle); SIN= sin(angle)
			if axis[2]==axis[3]==0.0    # rotation about x
				mat[2,2] = COS;    mat[2,3] = -SIN;
				mat[3,2] = SIN;    mat[3,3] = COS;
			elseif axis[1]==axis[3]==0.0   # rotation about y
				mat[1,1] = COS;    mat[1,3] = SIN;
				mat[3,1] = -SIN;    mat[3,3] = COS;
			elseif axis[1]==axis[2]==0.0    # rotation about z
				mat[1,1] = COS;    mat[1,2] = -SIN;
				mat[2,1] = SIN;    mat[2,2] = COS;
			else
				I = Matrix{Float64}(LinearAlgebra.I, 3, 3); u = axis
				Ux=[0 -u[3] u[2] ; u[3] 0 -u[1] ;  -u[2] u[1] 1]
				UU =[u[1]*u[1]    u[1]*u[2]   u[1]*u[3];
					 u[2]*u[1]    u[2]*u[2]   u[2]*u[3];
					 u[3]*u[1]    u[3]*u[2]   u[3]*u[3]]
				mat[1:3,1:3]=COS*I+SIN*Ux+(1.0-COS)*UU
			end
		end
	end
	return mat
end

r (generic function with 1 method)

In [44]:
@btime r(0)

  131.988 ns (2 allocations: 256 bytes)


3×3 Array{Float64,2}:
 1.0  -0.0  0.0
 0.0   1.0  0.0
 0.0   0.0  1.0

In [45]:
@btime r(1,1,1)

  834.556 ns (9 allocations: 1.41 KiB)


4×4 Array{Float64,2}:
 1.0      0.17353  2.14758  0.0
 2.14758  1.0      0.17353  0.0
 0.17353  2.14758  1.98703  0.0
 0.0      0.0      0.0      1.0

## Nuove funzioni

In [46]:
function r2D(args)
	angle = args[1]; COS = cos(angle); SIN = sin(angle)
	mat = Matrix{Float64}(LinearAlgebra.I, 3, 3)
	mat[1,1] = COS;    mat[1,2] = -SIN;
	mat[2,1] = SIN;    mat[2,2] = COS;
	return mat
end

r2D (generic function with 1 method)

In [47]:
function rX(mat,COS,SIN)
	mat[2,2] = COS;    mat[2,3] = -SIN;
	mat[3,2] = SIN;    mat[3,3] = COS;
	return mat
end

rX (generic function with 1 method)

In [48]:
function rY(mat,COS,SIN)
	mat[1,1] = COS;    mat[1,3] = SIN;
	mat[3,1] = -SIN;    mat[3,3] = COS;
	return mat
end

rY (generic function with 1 method)

In [49]:
function rZ(mat,COS,SIN)
	mat[1,1] = COS;    mat[1,2] = -SIN;
	mat[2,1] = SIN;    mat[2,2] = COS;
	return mat
end

rZ (generic function with 1 method)

In [50]:
function rAxis(mat,axis,COS,SIN)
	I = Matrix{Float64}(LinearAlgebra.I, 3, 3); u = axis
	Ux=[0 -u[3] u[2] ; u[3] 0 -u[1] ;  -u[2] u[1] 1]
	UU =[u[1]*u[1]    u[1]*u[2]   u[1]*u[3];
		 u[2]*u[1]    u[2]*u[2]   u[2]*u[3];
		 u[3]*u[1]    u[3]*u[2]   u[3]*u[3]]
	mat[1:3,1:3]=COS*I+SIN*Ux+(1.0-COS)*UU
	return mat
end

rAxis (generic function with 1 method)

In [51]:
function r3D(args)
	mat = Matrix{Float64}(LinearAlgebra.I, 4, 4)
	angle = norm(args);
	if angle != 0.0
		 axis = args #normalize(args)
		 COS = cos(angle); SIN= sin(angle)
		 if axis[2]==axis[3]==0.0    # rotation about x
			 mat = rX(mat,COS,SIN)
		 elseif axis[1]==axis[3]==0.0   # rotation about y
			 mat = rY(mat,COS,SIN)
		 elseif axis[1]==axis[2]==0.0    # rotation about z
			 mat = rZ(mat,COS,SIN)
		 else
			 mat = rAxis(mat,axis,COS,SIN)
		 end
	 end
	 return mat
end

r3D (generic function with 1 method)

In [52]:
function r(args...)
   n = length(args)

   if n == 1 # rotation in 2D
	   mat = r2D(args)
   end

    if n == 3 # rotation in 3D
       mat = r3D(args)
	end
	return mat
end

r (generic function with 1 method)

In [53]:
@btime r(0)

  87.527 ns (1 allocation: 160 bytes)


3×3 Array{Float64,2}:
 1.0  -0.0  0.0
 0.0   1.0  0.0
 0.0   0.0  1.0

In [54]:
@btime r(1,1,1)

  764.463 ns (8 allocations: 1.30 KiB)


4×4 Array{Float64,2}:
 1.0      0.17353  2.14758  0.0
 2.14758  1.0      0.17353  0.0
 0.17353  2.14758  1.98703  0.0
 0.0      0.0      0.0      1.0

In [55]:
@benchmark r(1,1,1)

BenchmarkTools.Trial: 
  memory estimate:  1.30 KiB
  allocs estimate:  8
  --------------
  minimum time:     759.504 ns (0.00% GC)
  median time:      784.298 ns (0.00% GC)
  mean time:        980.480 ns (2.26% GC)
  maximum time:     26.402 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     121

In [56]:
@code_typed r(1,1,1)

CodeInfo(
1 ─      goto #3 if not false
2 ─      nothing::Nothing
3 ┄      goto #5 if not true
4 ─ %4 = invoke Main.r3D(_2::Tuple{Int64,Int64,Int64})::Array{Float64,2}
5 ┄ %5 = φ (#4 => %4, #3 => #undef)::Core.Compiler.MaybeUndef(Array{Float64,2})
└──      return %5
) => Array{Float64,2}

In [57]:
@code_warntype r(1,1,1)

Variables
  #self#::Core.Compiler.Const(r, false)
  args::Tuple{Int64,Int64,Int64}
  n::Int64
  mat::Array{Float64,2}

Body::Array{Float64,2}
1 ─      Core.NewvarNode(:(mat))
│        (n = Main.length(args))
│   %3 = (n::Core.Compiler.Const(3, false) == 1)::Core.Compiler.Const(false, false)
└──      goto #3 if not %3
2 ─      Core.Compiler.Const(:(mat = Main.r2D(args)), false)
3 ┄ %6 = (n::Core.Compiler.Const(3, false) == 3)::Core.Compiler.Const(true, false)
└──      goto #5 if not %6
4 ─      (mat = Main.r3D(args))
5 ┄      return mat


## Modifiche

In [118]:
@inline function r2D(args)
	angle = args[1]; COS = cos(angle); SIN = sin(angle)
	mat = Matrix{Float64}(LinearAlgebra.I, 3, 3)
	mat[1,1] = COS;    mat[1,2] = -SIN;
	mat[2,1] = SIN;    mat[2,2] = COS;
	return mat
end

r2D (generic function with 2 methods)

In [119]:
@inline function rX(mat,COS,SIN)
	mat[2,2] = COS;    mat[2,3] = -SIN;
	mat[3,2] = SIN;    mat[3,3] = COS;
	return mat
end

rX (generic function with 1 method)

In [120]:
@inline function rY(mat,COS,SIN)
	mat[1,1] = COS;    mat[1,3] = SIN;
	mat[3,1] = -SIN;    mat[3,3] = COS;
	return mat
end

rY (generic function with 1 method)

In [121]:
@inline function rZ(mat,COS,SIN)
	mat[1,1] = COS;    mat[1,2] = -SIN;
	mat[2,1] = SIN;    mat[2,2] = COS;
	return mat
end

rZ (generic function with 1 method)

In [122]:
@inline @fastmath function rAxis(mat,axis,COS,SIN)
	I = Matrix{Float64}(LinearAlgebra.I, 3, 3); u = axis
	Ux=[0 -u[3] u[2] ; u[3] 0 -u[1] ;  -u[2] u[1] 1]
	UU =[u[1]*u[1]    u[1]*u[2]   u[1]*u[3];
         u[2]*u[1]    u[2]*u[2]   u[2]*u[3];
         u[3]*u[1]    u[3]*u[2]   u[3]*u[3]]
	mat[1:3,1:3]=COS*I+SIN*Ux+(1.0-COS)*UU
	return mat
end

rAxis (generic function with 1 method)

In [127]:
@inline function r3D(args)
	mat = Matrix{Float64}(LinearAlgebra.I, 4, 4)
	angle = norm(args);
	if angle != 0.0
		 axis = args #normalize(args)
		 COS = cos(angle); SIN= sin(angle)
		 if axis[2]==axis[3]==0.0    # rotation about x
			 mat = rX(mat,COS,SIN)
		 elseif axis[1]==axis[3]==0.0   # rotation about y
			 mat = rY(mat,COS,SIN)
		 elseif axis[1]==axis[2]==0.0    # rotation about z
			 mat = rZ(mat,COS,SIN)
		 else
			 mat = rAxis(mat,axis,COS,SIN)
		 end
	 end
	 return mat
end

r3D (generic function with 1 method)

In [124]:
@inline function r(args...)
   n = length(args)

   if n == 1 # rotation in 2D
	   mat = r2D(args)
   end

    if n == 3 # rotation in 3D
       mat = r3Ds(args)
	end
	return mat
end

r (generic function with 1 method)

In [74]:
@code_typed r(1,1,1)

CodeInfo(
1 ──        goto #3 if not false
2 ──        nothing::Nothing
3 ┄─        goto #31 if not true
4 ── %4   = LinearAlgebra.I::UniformScaling{Bool}
│    %5   = invoke Array{Float64,2}(%4::UniformScaling{Bool}, (4, 4)::Tuple{Int64,Int64})::Array{Float64,2}
│    %6   = Main.norm::typeof(norm)
│    %7   = invoke %6(_2::Tuple{Int64,Int64,Int64}, 2::Int64)::Float64
│    %8   = Base.ne_float(%7, 0.0)::Bool
└───        goto #29 if not %8
5 ── %10  = invoke Main.cos(%7::Float64)::Float64
│    %11  = invoke Main.sin(%7::Float64)::Float64
│    %12  = Base.getfield(args, 3, true)::Int64
│    %13  = Base.getfield(args, 2, true)::Int64
│    %14  = (%13 === %12)::Bool
└───        goto #7 if not %14
6 ── %16  = Base.sitofp(Float64, %12)::Float64
│    %17  = Base.eq_float(0.0, %16)::Bool
│    %18  = Base.ne_float(%16, 9.223372036854776e18)::Bool
│    %19  = Base.and_int(%17, %18)::Bool
│    %20  = Base.fptosi(Int64, %16)::Int64
│    %21  = (%12 === %20)::Bool
│    %22  = Base.and_int(%19, %21):

In [128]:
@btime r(1,1,1)

  689.467 ns (8 allocations: 1.30 KiB)


4×4 Array{Float64,2}:
 1.0      0.17353  2.14758  0.0
 2.14758  1.0      0.17353  0.0
 0.17353  2.14758  1.98703  0.0
 0.0      0.0      0.0      1.0

In [126]:
@btime r(0)

  76.028 ns (1 allocation: 160 bytes)


3×3 Array{Float64,2}:
 1.0  -0.0  0.0
 0.0   1.0  0.0
 0.0   0.0  1.0